In [295]:
import pandas as pd
import numpy as np

rki_infektion_landkreis_file = '../0_datasets/rki-corona-infektion-landkreis.txt'
rki_impfung_landkreis_file = "../0_datasets/rki-corona-impfung-landkreis.txt"
regionalschluessel_file = "../0_datasets/Regionalschluessel.txt"
wetterStationen_file = "../0_datasets/wetterStationen.txt"
wetterDaten_file = "../0_datasets/wetterDaten.txt"

def split(string_to_split):
    splits = [5, 14, 23, 38, 50, 61, 102]
    return_value = list()
    previous = 0
    for position in splits:
        value = string_to_split[previous:position].strip()
        if "(" in value:
            value = value.split("(")[0]
        if "," in value:
            split = value.split(",")
            value = split[1] + " " + split[0]
            value = value.replace("  ", " ").strip()
        return_value.append(value)
        previous = position
    return_value.append(string_to_split[previous:].replace(" ", "").replace("\n", ""))
    return return_value

def read_wetter(path):
    with open(path, "r", encoding="utf-8") as fp:
        lines = fp.readlines()
    header = lines[0]
    lines.remove(header)
    header = header.split()
    arr = np.zeros(shape=(len(lines),len(header)))
    simple_list=[]
    for line in lines:
        result = split(line)
        simple_list.append(result)
    dataframe = pd.DataFrame(simple_list, columns=header)
    return dataframe

In [333]:
#impfung
rki_impfung = pd.read_csv(rki_impfung_landkreis_file, low_memory=False)
rki_impfung = rki_impfung.drop(rki_impfung.columns[[2,3]], axis=1)
rki_impfung = rki_impfung.groupby(['Impfdatum', 'LandkreisId_Impfort'], as_index=False).sum()
rki_impfung = rki_impfung.rename(columns={'Anzahl': 'Impf_Anzahl', 'Impfdatum': 'Datum', 'LandkreisId_Impfort': 'ID'})
rki_impfung["Datum"] = rki_impfung["Datum"].map(lambda x: x.replace("-", ""))
rki_impfung["Datum"] = rki_impfung["Datum"].apply(pd.to_numeric)
rki_impfung["ID"] = rki_impfung["ID"].apply(pd.to_numeric)

#infektion
rki_infektion = pd.read_csv(rki_infektion_landkreis_file, low_memory=False)
rki_infektion = rki_infektion.drop(rki_infektion.columns[[1,2,3,5,6,7,8,10,11]], axis=1)
rki_infektion = rki_infektion.groupby(['Refdatum', 'IdLandkreis'], as_index=False).sum()
rki_infektion = rki_infektion.rename(columns={'AnzahlFall': 'Infek_Anzahl', 'Refdatum': 'Datum', 'IdLandkreis': 'ID'})
rki_infektion["Datum"] = rki_infektion["Datum"].map(lambda x: x.replace("-", ""))
rki_infektion["Datum"] = rki_infektion["Datum"].apply(pd.to_numeric)
rki_infektion["ID"] = rki_infektion["ID"].apply(pd.to_numeric)

#regionalschlüssel
regional_schluessel = pd.read_csv(regionalschluessel_file, low_memory=False)
regional_schluessel["ID"] = regional_schluessel["ID"].map(lambda x: int(x/10000000))
regional_schluessel = regional_schluessel.drop_duplicates(subset=['ID'], keep='first')
regional_schluessel = regional_schluessel.rename(columns={'Name': 'Ort'})
regional_schluessel[["ID"]] = regional_schluessel[["ID"]].apply(pd.to_numeric)

#wetterStationen
wetter_stationen = read_wetter(wetterStationen_file)
wetter_stationen = wetter_stationen[wetter_stationen["bis_datum"] == '20220608']
wetter_stationen = wetter_stationen.drop(wetter_stationen.columns[[1,2,3,4,5,7]], axis=1)
wetter_stationen = wetter_stationen.rename(columns={'Stations_id': 'ID', 'Stationsname': 'Ort'})
wetter_stationen[["ID"]] = wetter_stationen[["ID"]].apply(pd.to_numeric)

#wetterDaten
wetter_daten = pd.read_csv(wetterDaten_file, sep = ';', low_memory=False)
wetter_daten = wetter_daten.drop(wetter_daten.columns[[2,3,4,5,7,9,10,11,12,14,15,16,17,18]], axis=1)
wetter_daten[["MESS_DATUM"]] = wetter_daten[["MESS_DATUM"]].apply(pd.to_numeric)
wetter_daten = wetter_daten[wetter_daten["MESS_DATUM"] >= 20200119]
wetter_daten = wetter_daten.rename(columns={'STATIONS_ID': 'ID','MESS_DATUM': 'Datum', 'RSK': 'Regenmenge', 'SDK': 'Sonnenschein', 'TMK': 'Temperatur'})
wetter_daten[["ID"]] = wetter_daten[["ID"]].apply(pd.to_numeric)


In [335]:
#join
result = pd.merge(rki_infektion, rki_impfung, on=['Datum','ID'])
result = pd.merge(result, regional_schluessel, on=['ID'])
result = result.drop(result.columns[[1]], axis=1)

wetterResult = pd.merge(wetter_daten, wetter_stationen, on=['ID'])
wetterResult = wetterResult.drop(wetterResult.columns[[0]], axis=1)

result = pd.merge(result, wetterResult, on=['Datum', 'Ort'])

#result
with pd.option_context('display.max_rows', None):
    print(result)

        Datum  Infek_Anzahl  Impf_Anzahl        Ort  Regenmenge  Sonnenschein  \
0    20201229            25          232     Lübeck         0.7         0.017   
1    20201230            24          220     Lübeck         0.0         0.000   
2    20201231            24          286     Lübeck         0.0         0.000   
3    20210101            22          210     Lübeck         1.3         0.000   
4    20210102            26          170     Lübeck         1.5         0.000   
5    20210103            24          178     Lübeck         0.4         0.267   
6    20210104            30          414     Lübeck         3.9         0.000   
7    20210105            33          407     Lübeck         2.1         0.000   
8    20210106            31          393     Lübeck         8.3         0.000   
9    20210107            30          432     Lübeck         1.8         0.000   
10   20210108            41          392     Lübeck         6.5         0.000   
11   20210109            41 